In [1]:
using Interpolations, Plots, CUDA, Flux
using Plots

┌ Info: Precompiling Interpolations [a98d9a8b-a2ab-59e6-89dd-64a1c18fca59]
└ @ Base loading.jl:1278
┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1278


In [2]:
import Interpolations: degree, NoInterp, itpflag, tcollect

In [3]:
u = rand(5)
t = 0:0.1:4
t = repeat(t,1,5) .+ u'
U = cos.(π*t)|> gpu

41×5 CuArray{Float32,2}:
 -0.425697   0.514351  -0.299755     0.999214   -0.873898
 -0.68448    0.22417   -0.579891     0.938061   -0.981342
 -0.876263  -0.087954  -0.803264     0.785084   -0.992726
 -0.98227   -0.391469  -0.948007     0.555258   -0.906935
 -0.992126  -0.656664  -0.999953     0.271079   -0.732367
 -0.904866  -0.85758   -0.954016    -0.0396355  -0.48611
 -0.729031  -0.97455   -0.814694    -0.34647    -0.192269
 -0.481834  -0.996125  -0.595624    -0.619389    0.120393
 -0.187471  -0.920191  -0.31825     -0.831678    0.42127
  0.125242  -0.754184  -0.00972327  -0.962557    0.68091
  0.425697  -0.514351   0.299755    -0.999214    0.873898
  0.68448   -0.22417    0.579891    -0.938061    0.981342
  0.876263   0.087954   0.803264    -0.785084    0.992726
  ⋮                                             
  0.125242  -0.754184  -0.00972327  -0.962557    0.68091
  0.425697  -0.514351   0.299755    -0.999214    0.873898
  0.68448   -0.22417    0.579891    -0.938061    0.981342
  

In [4]:
itp = interpolate(U, (BSpline(Linear()), Tuple((NoInterp() for i in 2:ndims(U)))...));

In [5]:
itp

41×5 interpolate(::Array{Float32,2}, (BSpline(Linear()), NoInterp())) with element type Float32:
 -0.425697   0.514351  -0.299755     0.999214   -0.873898
 -0.68448    0.22417   -0.579891     0.938061   -0.981342
 -0.876263  -0.087954  -0.803264     0.785084   -0.992726
 -0.98227   -0.391469  -0.948007     0.555258   -0.906935
 -0.992126  -0.656664  -0.999953     0.271079   -0.732367
 -0.904866  -0.85758   -0.954016    -0.0396355  -0.48611
 -0.729031  -0.97455   -0.814694    -0.34647    -0.192269
 -0.481834  -0.996125  -0.595624    -0.619389    0.120393
 -0.187471  -0.920191  -0.31825     -0.831678    0.42127
  0.125242  -0.754184  -0.00972327  -0.962557    0.68091
  0.425697  -0.514351   0.299755    -0.999214    0.873898
  0.68448   -0.22417    0.579891    -0.938061    0.981342
  0.876263   0.087954   0.803264    -0.785084    0.992726
  ⋮                                             
  0.125242  -0.754184  -0.00972327  -0.962557    0.68091
  0.425697  -0.514351   0.299755    -0.999214 

In [14]:
t = 1.2 |>gpu
[Interpolations.gradient(itp, t, i) for i in 1:size(U,2)]

5-element Array{StaticArrays.SArray{Tuple{1},Float64,1,1},1}:
 [-0.27028998732566833]
 [-0.26327213644981384]
 [-0.3021206855773926]
 [-0.2634897828102112]
 [-0.307316891849041]

In [12]:

∂driving_data = zeros(size(itp,2))
 for i=1:size(itp,2)
   ∂driving_data[i] = Interpolations.gradient(itp, t, i)[1]
 end
 ∂driving_data

5-element Array{Float64,1}:
 -0.27028998732566833
 -0.26327213644981384
 -0.3021206855773926
 -0.2634897828102112
 -0.307316891849041

In [13]:
U

41×5 CuArray{Float32,2}:
 -0.362304   -0.402004   -0.105601   0.664314    0.338903
 -0.632594   -0.665276   -0.407721   0.400824    0.031586
 -0.840961   -0.863427   -0.669931   0.0980987  -0.278823
 -0.967009   -0.977059   -0.866564  -0.214229   -0.561938
 -0.9984     -0.995049   -0.978371  -0.505587   -0.790048
 -0.93206    -0.915638   -0.994409  -0.747454   -0.940821
 -0.774484   -0.746597   -0.913106  -0.916155   -0.999501
 -0.541095   -0.504474   -0.742423  -0.995177   -0.960343
 -0.254741   -0.21297    -0.499066  -0.976783   -0.827179
  0.0565491   0.0993812  -0.206857  -0.862776   -0.613046
  0.362304    0.402004    0.105601  -0.664314   -0.338903
  0.632594    0.665276    0.407721  -0.400824   -0.031586
  0.840961    0.863427    0.669931  -0.0980987   0.278823
  ⋮                                             
  0.0565491   0.0993812  -0.206857  -0.862776   -0.613046
  0.362304    0.402004    0.105601  -0.664314   -0.338903
  0.632594    0.665276    0.407721  -0.400824   -0.03158